# CASSANDRA STRUCTURES

In [45]:
#create KEYSPACE assignment3 WITH replication =  
#{'class': 'SimpleStrategy', 'replication_factor': 1};



#create table avgPastPop(
#               ... movieid text,
#               ... average decimal,
#               ... PRIMARY KEY(movieid)
#               ... );


#create table totEng(
#               ... time text,
#               ... total int,
#               ... PRIMARY KEY(time)
#               ... );

#create table engPerUser(
#               ... userid text,
#               ... eng int,
#               ... noneng int,
#               ... PRIMARY KEY(userid)
#               ... );


## IMPORTS

In [46]:
from pyspark.sql import functions as F
from datetime import datetime
from operator import add
from pyspark.sql import Row
from pyspark.streaming import StreamingContext
import time

## FUNCTIONS USED IN THE ASSIGNMENT

In [47]:
# function to return engagements (favorites + retweets)
def retEng(tweet):
    tweet_parts = tweet.split(',')
    sum = int(tweet_parts[4]) + int(tweet_parts[5])
    return sum


#function to return the average of past popularity and current popularity
def getAverage(time, rdd):
    print("========= %s =========" % str(time))
    try:
        rowRdd = rdd.map(lambda w: Row(movieid=w[0], average=w[1]))
        twitterDataFrame = spark.createDataFrame(rowRdd)
        twitterDataFrame.registerTempTable("twitterDF")
        twitterDF = sqlContext.sql("select tdf.movieid, ((tdf.average "+\
                                   "+ p.engagement) / 2) as average "+\
                                   "from twitterDF tdf join past p "+\
                                   "on tdf.movieid = p.movie")
        twitterDF.sort(F.desc("average")).show(20,False)
#code to write data in cassandra        
        twitterDF.write.format("org.apache.spark.sql.cassandra")\
        .options(table="avgpastpop", keyspace="assignment3")\
        .save(mode="append")
    except:
        pass

#function to calculate the total engagements recieved by movies in one batch
def getTotEng(time, rdd):
    print("========= %s =========" % str(time))
    try:
        Totdf = spark.createDataFrame(rdd.map(lambda row: Row(time = time, total=row[1])))
        Totdf.sort(F.desc("total")).show(20,False);
        
        Totdf.write.format("org.apache.spark.sql.cassandra")\
        .options(table="toteng", keyspace="assignment3")\
        .save(mode="append")
        
    except:
        pass

#function to calculate tweets with and without interactions, per user, per hour
def tweetsZeroNonZero(time, rdd):
    print("========= %s =========" % str(time))
    try:
        engagementRdd = rdd.filter(lambda row: row[1] > 0).map(lambda row: ((row[0][1]), 1))
        noEngagementRdd = rdd.filter(lambda row: row[1] == 0).map(lambda row: ((row[0][1]), 1))
        engagementByUser = engagementRdd.reduceByKey(lambda x, y: x + y)
        noEngagementByUser = noEngagementRdd.reduceByKey(lambda x, y: x + y)
        joinedRDD = engagementByUser.fullOuterJoin(noEngagementByUser)
        joinedRowRDD = joinedRDD.map(lambda w: Row(userid=w[0], eng=w[1][0], noneng=w[1][1]))
        
        tweetUserDF = spark.createDataFrame(joinedRowRDD)
        
        tweetUserDF.show()
        
        tweetUserDF.write.format("org.apache.spark.sql.cassandra")\
        .options(table="engperuser", keyspace="assignment3")\
        .save(mode="append")
    except:
        pass


## Past Data

In [35]:
dataset = spark.read.json("/root/twitter.medium")

In [36]:
dataset.registerTempTable("tweets")

In [37]:
movies_engagement = spark.sql("select entities.urls[0].display_url as movie, "+\
               "sum(favorite_count) + sum(retweet_count) as engagement "
               "from tweets "+\
               "group by entities.urls[0].display_url "+\
                "order by engagement DESC")

#using temp table to get engagements per movie

In [38]:
movies_engagement.registerTempTable("past")

## Task 3

In [48]:
#calculate the average past popularity of movies tweeted about within an hour
ssc = StreamingContext(sc, 10)

#creates a DStream of incoming batches
tweetRDD = ssc.textFileStream("file:///root/wcstream/")
tweetLineRDD = tweetRDD.flatMap(lambda tweet: tweet.split("|"))
tweetSepRDD = tweetLineRDD.map(lambda tweet: (tweet.split(",")[3]\
                                              ,retEng(tweet)))
tweetWindowRDD = tweetSepRDD.window(20, 10)

tweetReduceRDD = tweetWindowRDD.reduceByKey(lambda x, y: x + y)

In [49]:
tweetReduceRDD.foreachRDD(getAverage)

ssc.start()
time.sleep(40)
ssc.stop(False)

========= 2017-04-24 01:41:40 =========
========= 2017-04-24 01:41:50 =========
+------------------------+-------+
|movieid                 |average|
+------------------------+-------+
|imdb.com/title/tt1045658|9.5    |
|imdb.com/title/tt1623205|9.0    |
|imdb.com/title/tt0454876|7.5    |
|imdb.com/title/tt1659337|5.5    |
|imdb.com/title/tt1024648|5.5    |
|imdb.com/title/tt0840361|4.5    |
|imdb.com/title/tt0151804|3.5    |
|imdb.com/title/tt0903624|3.0    |
|imdb.com/title/tt1637725|2.5    |
|imdb.com/title/tt1853728|2.5    |
|imdb.com/title/tt2106476|2.5    |
|imdb.com/title/tt1673434|2.5    |
|imdb.com/title/tt0110912|2.5    |
|imdb.com/title/tt0111161|2.5    |
|imdb.com/title/tt1673697|2.0    |
|imdb.com/title/tt1371111|2.0    |
|imdb.com/title/tt2053463|2.0    |
|imdb.com/title/tt2101341|1.5    |
|imdb.com/title/tt0040746|1.5    |
|imdb.com/title/tt1702439|1.5    |
+------------------------+-------+
only showing top 20 rows

========= 2017-04-24 01:42:00 =========
+-------------

## Task 4

In [43]:
#calculate the total engagement recieved per hour

ssc = StreamingContext(sc, 10)

tweetRDD = ssc.textFileStream("file:///root/wcstream/")
tweetLinesRDD = tweetRDD.flatMap(lambda tweet : tweet.split("|"))
tweetSepRDD = tweetLinesRDD.map(lambda tweet: ("total engagement", retEng(tweet)))
tweetWindowRDD = tweetSepRDD.window(10, 10)

tweetReduceRDD = tweetWindowRDD.reduceByKey(lambda x, y: x + y)

In [44]:
tweetReduceRDD.foreachRDD(getTotEng)

ssc.start()
time.sleep(50)
ssc.stop(False)

========= 2017-04-24 00:52:40 =========
========= 2017-04-24 00:52:50 =========
+---------------------+-----+
|time                 |total|
+---------------------+-----+
|2017-04-24 00:52:50.0|109  |
+---------------------+-----+

========= 2017-04-24 00:53:00 =========
========= 2017-04-24 00:53:10 =========
+---------------------+-----+
|time                 |total|
+---------------------+-----+
|2017-04-24 00:53:10.0|177  |
+---------------------+-----+

========= 2017-04-24 00:53:20 =========
+---------------------+-----+
|time                 |total|
+---------------------+-----+
|2017-04-24 00:53:20.0|145  |
+---------------------+-----+



## Task 5

In [21]:
#number of user tweets with and without engagements, per user, per hour
ssc = StreamingContext(sc, 5)

lines = ssc.textFileStream("file:///root/wcstream/")
words = lines.flatMap(lambda line: line.split("|"))
pairs = words.map(lambda word: ((word.split(",")[0], word.split(",")[6]),retEng(word)))
pairsWindowed = pairs.window(20, 10)

wordCounts = pairsWindowed.reduceByKey(lambda x, y: x + y)

In [22]:
wordCounts.foreachRDD(tweetsZeroNonZero)

ssc.start()
time.sleep(60)
ssc.stop(False)

========= 2017-04-23 23:56:15 =========
========= 2017-04-23 23:56:25 =========
+----+------+---------------+
| eng|noneng|         userid|
+----+------+---------------+
|null|     1|       pixeldim|
|null|     1|     ShiftyGreg|
|null|     2|      sebutiban|
|   1|  null|        jensees|
|null|     3|patrickdconinck|
|null|     1|PeppermintShore|
|null|     2|     Jordi_lord|
|null|     2|   shereefislam|
|null|     5| ShahnazarShant|
|null|     1|      chropkins|
|null|     1|       jrsevans|
|null|     1|  Robiinderooij|
|null|    10|       _Afinso_|
|null|     1|    Caprini2029|
|null|     1|          I_Hmz|
|null|     2|        Uddin83|
|null|     1|      Danizio76|
|null|     8| mookieineugene|
|null|     1|       sengming|
|null|     2|    MyrvoldAlex|
+----+------+---------------+
only showing top 20 rows

========= 2017-04-23 23:56:35 =========
+----+------+---------------+
| eng|noneng|         userid|
+----+------+---------------+
|null|     1|       pixeldim|
|null|     1| 